In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

# The Fishing dataset is in the 'mlogit' R package
dataset_link = "https://vincentarelbundock.github.io/Rdatasets/csv/Ecdat/Fishing.csv"
raw_df = pd.read_csv(dataset_link)

raw_df

,rownames,mode,price,catch,pbeach,ppier,pboat,pcharter,cbeach,cpier,cboat,ccharter,income
0,1,charter,182.930,0.5391,157.930,157.930,157.930,182.930,0.0678,0.0503,0.2601,0.5391,7083.33170
1,2,charter,34.534,0.4671,15.114,15.114,10.534,34.534,0.1049,0.0451,0.1574,0.4671,1249.99980
2,3,boat,24.334,0.2413,161.874,161.874,24.334,59.334,0.5333,0.4522,0.2413,1.0266,3749.99990
3,4,pier,15.134,0.0789,15.134,15.134,55.930,84.930,0.0678,0.0789,0.1643,0.5391,2083.33320
4,5,boat,41.514,0.1082,106.930,106.930,41.514,71.014,0.0678,0.0503,0.1082,0.3240,4583.33200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,1178,pier,150.236,0.4522,150.236,150.236,250.746,285.746,0.5333,0.4522,0.2413,1.0266,3749.99990
1178,1179,beach,235.436,0.5333,235.436,235.436,392.946,427.946,0.5333,0.4522,0.2413,1.0266,6250.00130
1179,1180,pier,65.036,0.4522,65.036,65.036,108.546,143.546,0.5333,0.4522,0.2413,1.0266,1249.99980
1180,1181,beach,36.636,0.5333,36.636,36.636,61.146,96.146,0.5333,0.4522,0.1665,0.3975,416.66668


In [12]:
X, y = raw_df.assign(
    charter_dummy = (raw_df['mode'] == 'charter').astype(int)
).pipe(
    lambda df: (
        df[['pbeach', 'pboat', 'pcharter', 'cbeach', 'cpier', 'cboat', 'ccharter', 'income']],
        df['charter_dummy']
    )
)

print("Value Counts for y:\n{}".format(y.value_counts()))
X


Value Counts for y:
charter_dummy
0    730
1    452
Name: count, dtype: int64


,pbeach,pboat,pcharter,cbeach,cpier,cboat,ccharter,income
0,157.930,157.930,182.930,0.0678,0.0503,0.2601,0.5391,7083.33170
1,15.114,10.534,34.534,0.1049,0.0451,0.1574,0.4671,1249.99980
2,161.874,24.334,59.334,0.5333,0.4522,0.2413,1.0266,3749.99990
3,15.134,55.930,84.930,0.0678,0.0789,0.1643,0.5391,2083.33320
4,106.930,41.514,71.014,0.0678,0.0503,0.1082,0.3240,4583.33200
...,...,...,...,...,...,...,...,...
1177,150.236,250.746,285.746,0.5333,0.4522,0.2413,1.0266,3749.99990
1178,235.436,392.946,427.946,0.5333,0.4522,0.2413,1.0266,6250.00130
1179,65.036,108.546,143.546,0.5333,0.4522,0.2413,1.0266,1249.99980
1180,36.636,61.146,96.146,0.5333,0.4522,0.1665,0.3975,416.66668


In [3]:
result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.627697
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          charter_dummy   No. Observations:                 1182
Model:                          Logit   Df Residuals:                     1174
Method:                           MLE   Df Model:                            7
Date:                Wed, 21 May 2025   Pseudo R-squ.:                 0.05642
Time:                        16:07:35   Log-Likelihood:                -741.94
converged:                       True   LL-Null:                       -786.30
Covariance Type:            nonrobust   LLR p-value:                 2.261e-16
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pbeach         0.0053      0.001      6.040      0.000       0.004       0.007
pboat         -0.0089      0.006     -1.561      0.119      -0.020       0.002
pcharter       0.0048      0.005      0.883      0.377      -0.006       0.016
cbeach        -0.1844      0.600     -0.307      0.758      -1.360       0.991
cpier         -1.3335      0.674     -1.980      0.048      -2.654      -0.013
cboat         -2.3521      0.879     -2.675      0.007      -4.076      -0.629
ccharter       0.8531      0.265      3.220      0.001       0.334       1.372
income        -0.0002   3.69e-05     -5.460      0.000      -0.000      -0.000
==============================================================================
"""

In [5]:
# Compute MEM, AME, and OR and interpret your results.

# MEM
mem = result.get_margeff(at='mean').summary()
print("\nMarginal Effects at the Mean (MEM):\n", mem)



Marginal Effects at the Mean (MEM):
         Logit Marginal Effects       
Dep. Variable:          charter_dummy
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pbeach         0.0012      0.000      6.105      0.000       0.001       0.002
pboat         -0.0021      0.001     -1.553      0.121      -0.005       0.001
pcharter       0.0011      0.001      0.880      0.379      -0.001       0.004
cbeach        -0.0434      0.141     -0.308      0.758      -0.320       0.233
cpier         -0.3141      0.159     -1.977      0.048      -0.625      -0.003
cboat         -0.5540      0.207     -2.681      0.007      -0.959      -0.149
ccharter       0.2009      0.062      3.229      0.001       0.079       0.323
income     -4.749e-05   8.61e-06     -5.517      0.000   -6.44e-05   -3.06e-05


In [6]:

# AME
ame = result.get_margeff(at='overall').summary()
print("\nAverage Marginal Effects (AME):\n", ame)



Average Marginal Effects (AME):
         Logit Marginal Effects       
Dep. Variable:          charter_dummy
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pbeach         0.0012      0.000      6.400      0.000       0.001       0.002
pboat         -0.0020      0.001     -1.565      0.118      -0.004       0.000
pcharter       0.0011      0.001      0.884      0.377      -0.001       0.003
cbeach        -0.0405      0.132     -0.307      0.758      -0.299       0.218
cpier         -0.2928      0.147     -1.993      0.046      -0.581      -0.005
cboat         -0.5165      0.191     -2.708      0.007      -0.890      -0.143
ccharter       0.1873      0.057      3.277      0.001       0.075       0.299
income     -4.427e-05   7.75e-06     -5.716      0.000   -5.95e-05   -2.91e-05


In [7]:

# OR
or_ = np.exp(result.params)
print("\nOdds Ratios (OR):\n", or_)




Odds Ratios (OR):
 pbeach      1.005288
pboat       0.991153
pcharter    1.004847
cbeach      0.831617
cpier       0.263544
cboat       0.095165
ccharter    2.346937
income      0.999798
dtype: float64
